In [1]:
from astroquery.jplhorizons import Horizons
from astroquery.jplsbdb import SBDB
from astropy.table import QTable
import astropy.units as u
import re

import pandas as pd
import numpy as np
import requests
from pprint import pprint
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [28]:
def get_horizons_ephemerides(name,pov,epoch_start,epoch_stop,step_size,idx_elements):
    
    # step: step size, [10m, 1d, 1y]
    
    if pov.lower() == 'sun':
        loc = '500@10' # position relative to the sun
    elif pov.lower() == 'goldstone':
        loc = '257' # from goldstone
    elif pov.lower() == 'maunakea':
        loc = '568' # maunakea
    else:
        print('Not Valid Location Point Of View')
    
    # Process to get homogeneity from main script full name '2012QD8' to a valid name for Horizon call '2012 QD8'
    if len(re.findall('([0-9])', name)) <= 4: # 4 is the min numbers in every name, the date year of discovery
        r = re.compile("([0-9]+)([a-zA-Z]+)").match(name)
        k1 = r.group(1) # the date of the name
        k2 = r.group(2) # the code of the date
        valid_name = k1 + " " + k2 
    else:
        r = re.compile("([0-9]+)([a-zA-Z]+)([0-9]+)").match(name)
        k1 = r.group(1) # the date of the name
        k2 = r.group(2) # the code of the date
        k3 = r.group(3) # id after the letters
        valid_name = k1 + " " + k2 + k3
    
    obj = Horizons(id=valid_name, 
               location=loc, 
               epochs={'start': epoch_start, 'stop':epoch_stop,
                       'step': step_size})
    
    if idx_elements.lower() == 'vectors':
        data = obj.vectors() # vectorial elements
        
        len_rows = len(data)
        len_cols = 6 # 3 positions 'x','y','z', and 3 velocities 'vx', 'vy', 'vz'
        idx_x = 5 # 'x' is at position 5 in the table (starting from 0)
        adata =  np.zeros([len_rows,len_cols]) 
        for row in range(len_rows):
            for col in range(6): 
                idx_col_in_table = idx_x + col # because the 'x' start at 6th col, going up till the 12th that is 'vz'
                adata[row,col] = data[row][idx_col_in_table]

    elif idx_elements.lower() == 'elements':
        # refsystem = 'J2000', # Element reference system for geometric and astrometric quantities
        # refplane = 'ecliptic' #ecliptic and mean equinox of reference epoch
        data = obj.elements(refsystem = 'J2000',refplane = 'ecliptic')
        
        len_rows = len(data)
        len_cols = 6 # a e i OM om theta
        adata =  np.zeros([len_rows,len_cols]) 
        for row in range(len_rows):
            adata[row,0] = data[row][14] # 15th column of data -> semimajor axis
            adata[row,1] = data[row][5] # 6th column of data -> eccentricity
            adata[row,2] = data[row][7] # 8th column of data -> inclination
            adata[row,3] = data[row][8] # 9th column of data -> RAAN, (OMEGA)
            adata[row,4] = data[row][9] # 10th column of data -> argument of perigee, (omega)
            adata[row,5] = data[row][13] # 14th column of data -> True anomaly, (theta)
        
    return adata

In [27]:
name = '2012QD8'
PointOfView = 'Sun'
epoch_start = '2021-01-01'
epoch_stop = '2024-01-01'
step = '5d'
idx_elements = 'elements'

data_2012QD8 = get_horizons_ephemerides(name,PointOfView,epoch_start,epoch_stop,step,idx_elements)
data_2012QD8

array([[  1.91262037,   0.69786667,   5.58072725, 348.73733034,
         87.77257299, 117.43366793],
       [  1.91261207,   0.69786616,   5.58073197, 348.73735439,
         87.77234221, 119.68499269],
       [  1.91260387,   0.69786576,   5.58073674, 348.73738101,
         87.77209956, 121.78355101],
       ...,
       [  1.91260623,   0.69808093,   5.5795977 , 348.72702449,
         87.76150114, 148.95867558],
       [  1.9126122 ,   0.69808117,   5.57958008, 348.72674402,
         87.76197244, 149.76792297],
       [  1.91261769,   0.69808135,   5.5795634 , 348.72647032,
         87.76242522, 150.55742091]])

In [ ]:
def get_earth_ephemerides(epoch_start,epoch_stop,step_size,idx_elements):
    
    # step: step size, [10m, 1d, 1y]

    obj = Horizons(id = 'Geocenter', 
               location = '500@10', 
               epochs = {'start': epoch_start, 'stop':epoch_stop,
                       'step': step_size},
               id_type = 'majorbody')
    
    if idx_elements.lower() == 'vectors':
        data = obj.vectors() # vectorial elements
    elif idx_elements.lower() == 'ephemerides':
        data = obj.ephemerides()
    
    len_rows = len(data)
    len_cols = 6 # 3 positions 'x','y','z', and 3 velocities 'vx', 'vy', 'vz'
    idx_x = 3 # 'x' is at position 3 in the table
    adata =  np.zeros([len_rows,len_cols]) 
    for row in range(len_rows):
        for col in range(6): 
            idx_col_in_table = idx_x + col # because the 'x' start at 3rd col, going up till the 9th that is 'vz'
            adata[row,col] = data[row][idx_col_in_table]

    return adata

In [ ]:
# now it doesn't work anymore because the data structure has changed in a numpy nd array
def plot_orbits(data,kolor):
    X = data['x']
    Y = data['y']
    Z = data['z']
    
    #fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    #ax.set_box_aspect((np.ptp(X), np.ptp(Y), np.ptp(Z))) # to set equivalent to axis equal
    ax.plot(X,Y,Z, label=data['targetname'][0], color=kolor)
    ax.legend()
    
    ax.xaxis.set_rotate_label(False) 
    ax.yaxis.set_rotate_label(False) 
    ax.zaxis.set_rotate_label(False) 
    ax.set_xlabel('$X$ [AU]')
    ax.set_ylabel('$Y$ [AU]')
    #ax.yaxis._axinfo['label']['space_factor'] = 3.0
    # set z ticks and labels
    #ax.set_zticks([-2, 0, 2])
    # disable auto rotation
    ax.set_zlabel('$Z$ [AU]')
    
    return ax

In [ ]:
# Earth Basic ephemerides
epoch_start = '2021-01-01'
epoch_stop = '2022-01-01'
step = '5d'
idx_elements = 'Vectors'

data_Earth = get_earth_ephemerides(epoch_start,epoch_stop,step,idx_elements)
data_Earth

In [ ]:
name = '2012QD8'
PointOfView = 'Sun'
epoch_start = '2021-01-01'
epoch_stop = '2024-01-01'
step = '5d'
idx_elements = 'Vectors'

data_2012QD8 = get_horizons_ephemerides(name,PointOfView,epoch_start,epoch_stop,step,idx_elements)
data_2012QD8

In [ ]:
fig = plt.figure()
ax_Earth = plot_orbits(data_Earth,'gray')
ax_2012QD8 = plot_orbits(data_2012QD8,'blue')
#plt.show()


In [ ]:
asteroids = ['2012QD8','2005WG57','2012BY1','2012SY49','2008XU2','2008KN11','2020UE','2006HX57','2006SC']

PointOfView = 'Sun'
epoch_start = '2021-01-01'
epoch_stop = '2025-01-01'
step = '10d'
idx_elements = 'Vectors'

#colors = ['black','aqua','green','orange','yellow']
data = {}
j=0

for ast in asteroids:
    elements = get_horizons_ephemeris(asteroids[j],PointOfView,epoch_start,epoch_stop,step,idx_elements)
    data_dict = {"fullname": elements['targetname'],
                 "x": elements['x'],
                 "y": elements['y'],
                 "z": elements['z'],
                };
    data[j]=data_dict
    j = j+1
    fig = plt.figure()
    plot_orbits(data_Earth,'gray')
    plot_orbits(elements,'blue')
    save_name = elements['targetname'][0]
    plt.savefig(f'./Figures/Orbits/{save_name}.png',bbox_inches='tight')